# Plot individual images on the cluster

This notebook explores how to view geospatial data on the cluster without needing to transfer it

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import datetime

import rasterio
from rasterio import plot
from rasterio.plot import show
import matplotlib.pyplot as plt
import shutil
import tempfile
import json
import random
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from pyproj import CRS
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
import xarray as xr

%matplotlib inline

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Basic Parameters: \n PrintDate = {} \n brdf_dir = {} \n data source = {} \n gridCell = {} \n index_dir = {} \n out_dir = {} \n plotYr = {}"
      .format(basicConfig['today'],basicConfig['brdf_dir'],basicConfig['data_source'],basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['out_dir'], basicConfig['plotYr']))

%store -r SinglePlotParams
print("Plotting Parameters: \n plotDay = {} \n Viewband = {} \n imageType = {}"
      .format(SinglePlotParams['plotDay'],SinglePlotParams['Viewband'],SinglePlotParams['imageType']))

In [ ]:
def GetImgDate(img, imageType, data_source):
    
    if imageType == 'Smooth':  #Expects images to be named YYYYDDD
        YYYY = int(img[:4])
        doy = int(img[4:7])
    elif imageType not in ['Sentinel','Landsat','AllRaw']:
        print ('Currently valid image types are Smooth,Sentinel,Landsat and AllRaw. You put {}'.format(imageType))
    elif data_source == 'GEE':
        if img.startswith('L1C'):
            YYYY = int(img[19:23])
            MM = int(img[23:25])
            DD = int(img[25:27])
        elif img.startswith('LC') or img.startswith('LT') or img.startswith('LE'):
            YYYY = int(img[17:21])
            MM = int(img[21:23])
            DD = int(img[23:25])
    elif data_source == 'stac':
        YYYY = int(img[-24:-20])
        MM = int(img[-20:-18])
        DD = int(img[-18:-16])
        
    ymd = datetime.datetime(YYYY, MM, DD)
    doy = int(ymd.strftime('%j'))
        
    return YYYY, doy

def GetClosestImage(img_dir, imageType, data_source, TargetYr, TargetDay):
    '''
    returns path of image closest to {TargetDay (DDD)} of {TargetYear (YYYY)} in directory of images, {img_dir}
    Currently looks for images with name YYYYDDD* (DDD is day-of-year(1-365)) if imageType = Smooth
    or Sentinel images starting with L1C* (Date format is YYYYMMDD, starting at position 19)
    '''
    imgFiles = []
    imgList = []
    if imageType == 'Smooth':
        for img in os.listdir(img_dir):
            if img.endswith('.tif'):
                imgDate = GetImgDate(img, 'Smooth', data_source)
                if imgDate[0] == TargetYr:
                    imgList.append(imgDate[1])
    else:
        for img in os.listdir(img_dir):
            if data_source == 'GEE':
                imgTyp = os.path.basename(img)[:2]
            elif data_source == 'stac':
                imgTyp = os.path.basename(img)[4:6]

            if (imageType in ['Sentinel','AllRaw'] and imgTyp in ['S2','L1']) or (imageType in ['Landsat','AllRaw'] and imgTyp in ['LC','LT','LE']):
                if img.endswith('.nc') and 'angles' not in img and 'cloudless' not in img:
                    imgFiles.append(img)
                    imgDate = GetImgDate(img, imageType, data_source)
                    if imgDate[0] == TargetYr:
                        imgList.append(imgDate[1])
                
    if len(imgList) == 0: 
        print('there are no {} images for target year {}'.format(imageType, TargetYr))
        
    else:       
        closestDay = min(imgList, key=lambda x:abs(x-TargetDay))
        print('closest day for {} is: {}'.format(imageType, str(TargetYr)+str(closestDay)))
        closestMM = (datetime.datetime(TargetYr, 1, 1) + datetime.timedelta(closestDay - 1)).month
        closestDD = (datetime.datetime(TargetYr, 1, 1) + datetime.timedelta(closestDay - 1)).day
    
        validImgs = [] #There can be more than one image if more than one path overlaps cell; pick the largest file
        for fname in imgFiles:
            if imageType == 'Smooth':
                if str(TargetYr)+str(closestDay) in fname:
                    validImgs.append(os.path.join(img_dir,fname))
            else:
                if str(TargetYr)+'{:02d}'.format(closestMM)+'{:02d}'.format(closestDD) in fname and fname.endswith('.nc') and 'angles' not in fname and 'cloudless' not in fname:
                    imgPath = os.path.join(img_dir,fname)
                    print('image match with {} of file size {}'.format(fname, os.stat(imgPath).st_size))
                    validImgs.append(imgPath)
                    
        closestimg = max(validImgs, key =  lambda x: os.stat(x).st_size)
        print('sample {} image is: {} with file size {}'.format(imageType, closestimg, os.stat(closestimg).st_size))
            
    return closestimg

In [ ]:
#SampImg_Smooth = GetClosestImage(basicConfig['index_dir'], 'Smooth', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
SampImg_Sentinel = GetClosestImage(basicConfig['brdf_dir'], 'Sentinel', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])
#SampImg_Landsat = GetClosestImage(basicConfig['brdf_dir'], 'Landsat', basicConfig['data_source'], basicConfig['plotYr'], SinglePlotParams['plotDay'])

In [ ]:
### For visualizing index image (Process check 1)
def exploreBand(img, band):

    print('plotting {} band of image {}'.format(band, img))
    fig, axarr = plt.subplots(1, 3, figsize=(15,5))
    
    if img.endswith('.tif'):
        print ('this is in .tif format')
        with rasterio.open(img) as src:
            TSamp = src.read()
    elif img.endswith('.nc'):
        print ('this is in .nc format')
        with xr.open_dataset(img) as xrimg:
            xrcrs = xrimg.crs
            print(xrcrs)  ##epsg:32632
            #print(xrimg.variables)
            bands=[i for i in xrimg.data_vars]
            print(bands)
            #print(xrimg.coords)
            
        ### Check that x and y values correspond to UTM coords
        print("Coord range is: y: {}-{}. x: {}-{}".format(
          xrimg[band]["y"].values.min(), 
          xrimg[band]["y"].values.max(),
          xrimg[band]["x"].values.min(), 
          xrimg[band]["x"].values.max()))
    
        ### Get single band (Opens as Dataset)
        xr_idx = xrimg[band]
    
        ##Note outliers (while nodata = None)
        origHist = xr_idx.plot.hist(color="purple")
        #ax.imshow()
        
        xr_idx_clean = xr_idx.where(xr_idx < 10000)
        maskedHist = xr_idx_clean.plot.hist(color="purple")
        #print('The no data value is:', xrimg[spec_index].rio.nodata)
    
        TSamp = xr_idx_clean

    if img.endswith('.nc'):
        xr_idx.plot.hist(color="purple",  ax=axarr[0])
        axarr[0].set_title("original data")
        xr_idx_clean.plot.hist(color="purple", ax=axarr[1])
        axarr[1].set_title("masked data")
        TSamp.plot(x="x",y="y")
        plot.show(TSamp, ax=axarr[2])
        axarr[2].set_title('{} band'.format(band))
        axarr[2].axis('off')
    
    elif img.endswith('.tif'):
        plot.show(TSamp)
        axarr[2].set_title('{} band'.format(band))
        axarr[2].axis('off')
        
    return axarr

In [ ]:
if SinglePlotParams['imageType'] == 'Smooth':
    exploreBand(SampImg_Smooth, basicConfig['spec_index'])
elif SinglePlotParams['imageType'] in ['Sentinel','AllRaw']:
    exploreBand(SampImg_Sentinel, SinglePlotParams['Viewband'])
elif SinglePlotParams['imageType'] == 'Landsat':
    exploreBand(SampImg_Landsat, SinglePlotParams['Viewband'])

## Plot 3-band image

In [ ]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

def gammacorr(band, gamma):
    return np.power(band, 1/gamma)

def GetrbgImg(image):
    with xr.open_dataset(image) as xrimg:
        red = xrimg['red'].where(xrimg['red'] != 65535)
        green = xrimg['green'].where(xrimg['green'] != 65535)
        blue = xrimg['blue'].where(xrimg['blue'] != 65535)
        #print (red.min(), red.max())
    
    red_g=gammacorr(red, SinglePlotParams['gamma'])
    blue_g=gammacorr(blue, SinglePlotParams['gamma'])
    green_g=gammacorr(green, SinglePlotParams['gamma'])

    red_norm_g = normalize(red_g)
    green_norm_g = normalize(green_g)
    blue_norm_g = normalize(blue_g)

    rgb = np.dstack([red_norm_g, green_norm_g, blue_norm_g])
    fig = plt.figure(figsize=(18,12))
    plt.imshow(rgb)

In [ ]:
GetrbgImg(SampImg_Sentinel)
plt.title("Sentinel")
plt.axis('off');

In [ ]:
GetrbgImg(SampImg_Landsat)
plt.title("Landsat")
plt.axis('off');

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

outName = str(basicConfig['country']+'1b_SampleImagePlot_for_Cell'+str(basicConfig['gridCell'])+'_from_'+str(SinglePlotParams['plotYr'])+str(SinglePlotParams['plotDay']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1b_ExploreData_OpenImage.ipynb